# Installation and Setup

In [142]:
!pip install gensim sentence-transformers

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [143]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
import pandas as pd
import nltk

# Import Dataset

df_courses berasal dari dataset dari tugas sebelumnya. df_query merupakan dummy data yang digenerate AI yang berisi query/request dari user untuk dicarikan course yang cocok untuknya.

In [144]:
df_courses = pd.read_csv('edx_courses.csv')
df_query = pd.read_csv('dummy_user_requests.csv')

# Data Preparation

Hasil preprocessing sebelumnya sama dengan apa yang sudah dilakukan di tugas pertama.

In [145]:
nltk.download('wordnet')
nltk.download('stopwords')

stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
stop_words = stopwords.words('english')

def lowering(text: str) -> str:
    text = text.lower()
    return text

def remove_punctuation_and_symbol(text: str) -> str:
    text = re.sub(r'[^\w\s]', '', text)
    return text

def stopword_removal(text: str) -> str:
    text = " ".join([word for word in text.split() if word not in stop_words])
    return text

# def stemming(text: str) -> str:
#     text = " ".join([stemmer.stem(word) for word in text.split()])
#     return text

def lemmatization(text: str) -> str:
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

def remove_html_tags(text: str) -> str:
    text = re.sub(r'<[^>]+>', '', text)
    return text

def preprocessing(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = remove_html_tags(text)
    text = lowering(text)
    text = remove_punctuation_and_symbol(text)
    text = stopword_removal(text)
    text = lemmatization(text)
    return text

[nltk_data] Downloading package wordnet to /home/lenovo/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/lenovo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [146]:
exclude_cols = [
    'marketing_url', 'availability', 'subject', 'level', 'product', 'program_type',
    'staff', 'translation_language', 'transcription_language', 'partner', 'title'
]

for col in df_courses.columns:
    if df_courses[col].dtype == 'object' and col not in exclude_cols:
        df_courses[col] = df_courses[col].apply(preprocessing)
        df_courses[col] = df_courses[col].replace("", None)

In [147]:
import ast

list_cols = [
    'availability', 'subject', 'level', 'program_type',
    'staff', 'translation_language', 'transcription_language', 'partner'
]

for col in list_cols:
    if col in df_courses.columns:
        df_courses[col] = df_courses[col].apply(
            lambda x: (
                ', '.join(ast.literal_eval(x)) if isinstance(x, str) and x.startswith('[') and x.strip() != "[]" else
                None if x == "" or x == "[]" else
                x
            )
        )
        df_courses[col] = df_courses[col].str.lower()
        
df_courses

,title,partner,primary_description,secondary_description,tertiary_description,availability,subject,level,language,product,program_type,staff,translation_language,transcription_language,recent_enrollment_count,marketing_url,weeks_to_complete,skill
0,How to Learn Online,edx,course prepare strategy successful online lear...,history benefit foundational concept online le...,course harness sciencebacked technique applica...,available now,education & teacher training,introductory,english,Course,None,None,"arabic, english, spanish (latin america), indo...","portuguese - brazil, indonesian, arabic, spanish",47714,https://www.edx.org/learn/how-to-learn/edx-how...,2.0,learning design
1,Corporate Finance,columbia university,learn sound theoretical principle finance prac...,learn value firm course corporate finance prof...,understand sound theoretical principle finance...,available now,"business & management, economics & finance",introductory,english,Program,professional certificate,daniel-wolfenzon,None,None,11648,https://www.edx.org/certificates/professional-...,NaN,cost capital finance investment banking invest...
2,The Science of Happiness,"university of california, berkeley",first mooc teach positive psychology learn sci...,happiness really mean matter increase happines...,free eightweek science happiness course offer ...,"upcoming, available now",social sciences,introductory,english,Course,None,"dacher-keltner, emiliana-simon-thomas","arabic, english, spanish (latin america), indo...","russian, indonesian, spanish, arabic, portugue...",16821,https://www.edx.org/learn/happiness/university...,11.0,empathy evolutionary biology psychology
3,Supply Chain Management,massachusetts institute of technology,earn micromasters credential mit 1 ranked supp...,gain expertise growing field supply chain mana...,apply core methodology probability statistic o...,"archived, upcoming, available now","business & management, engineering","advanced, introductory, intermediate",english,Program,micromasters,"yossi-sheffi, james-blayney-rice-jr, jarrod-go...",None,None,30033,https://www.edx.org/masters/micromasters/mitx-...,NaN,supply chain management innovation supply chain
4,Remote Work Revolution for Everyone,harvard university,remote work revolution everyone learn excel vi...,understand key element remote work use knowled...,thriving surviving remote work environment rem...,available now,business & management,introductory,english,Course,professional certificate,tsedal-neeley,"arabic, english, spanish (latin america), indo...","spanish, arabic, thai, korean, telugu, turkish...",45954,https://www.edx.org/learn/remote-work/harvard-...,3.0,telecommuting customer relationship building
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Eaux, villes et changements climatiques",université de montréal,vous vous intéressez aux enjeux de leau en con...,dans ce cours vous apprendrez à reconnaître le...,vous avez à cœur la protection de leau vous so...,available now,"engineering, environmental studies",introductory,french,Course,None,"louise-millette-2, veronique-gisondi",None,None,372,https://www.edx.org/learn/engineering/universi...,6.0,None
996,Fixed Income Portfolio Management,new york institute of finance,learn construct manage fixed income portfolio ...,review fixed income concept passive fixed inco...,fixed income portfolio management comprehensiv...,available now,economics & finance,advanced,english,Course,professional certificate,"chris-thomas, jack-farmer","arabic, english, spanish (latin america), indo...",None,371,https://www.edx.org/learn/portfolio-management...,4.0,portfolio management professional portfolio ma...
997,Artes na Educação,fundação para a ciência e a tecnologia,educação encontrase num profundo processo de m...,identificar o principais contributos da artes ...,sabemos que gerações mais nova estão sujeitas ...,available now,"education & teacher training, art & culture",introductory,portuguese,Course,None,"teresa-

In [148]:
df_courses.isnull().sum()

title                        0
partner                      0
primary_description          6
secondary_description        0
tertiary_description         1
availability                 0
subject                      0
level                        0
language                     0
product                      0
program_type               383
staff                       58
translation_language       245
transcription_language     400
recent_enrollment_count      0
marketing_url                0
weeks_to_complete           21
skill                      160
dtype: int64

Di sini, kolom yang digunakan untuk vektorisasi nanti hanyalah full_description, subject, dan skill

In [149]:
df_courses['full_description'] = df_courses['primary_description'].fillna('') + ' ' + \
                                df_courses['secondary_description'].fillna('') + ' ' + \
                                df_courses['tertiary_description'].fillna('')


In [150]:
df_courses['combined_text'] = df_courses['subject'] + ' ' + \
                      df_courses['skill'].fillna('') + ' ' + \
                      df_courses['full_description']


Nanti sistem rekomendasi ini akan diurutkan tidak hanya dari hasil similarity terbesar, tetapi juga dari tingkat level.

In [151]:
def custom_level_rank(level_str):
    level_str = level_str.lower()
    if 'introductory' in level_str:
        return 0
    elif 'intermediate' in level_str:
        return 1
    elif 'advanced' in level_str:
        return 2
    else:
        return 99
    
df_courses['level_rank'] = df_courses['level'].apply(custom_level_rank)

In [152]:
df_query['user_request'] = df_query['user_request'].apply(preprocessing)

# Vectorization

Penggunaan Sentence Transformers dalam proyek ini bertujuan untuk menghasilkan representasi vektor (embedding) dari teks yang lebih kontekstual dan bermakna, khususnya untuk teks dalam bentuk kalimat atau paragraf seperti deskripsi kursus dan permintaan pengguna.

In [153]:
# Inisialisasi model pretrained BERT
model_bert = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encode text kursus menjadi vektor
course_vectors = model_bert.encode(df_courses['combined_text'])

# Menampilkan matriks fitur dalam bentuk array
print("\nMatriks BERT:")
print(course_vectors)

# Hasil vektor dijadikan csv
df_vectors = pd.DataFrame(course_vectors)
df_vectors['title'] = df_courses['title'].values
df_vectors.to_csv("course_vectors.csv", index=False)


Matriks BERT:
[[-0.02670429  0.35512123  0.045985   ... -0.47753912 -0.06232988
   0.20351861]
 [-0.0023166   0.01453904 -0.12253136 ... -0.41341904 -0.11627714
   0.19310197]
 [-0.54057205 -0.04725458  0.22221217 ... -0.10978574 -0.20745039
   0.30415684]
 ...
 [ 0.2601171   0.21126032  0.30086675 ... -0.30056527  0.35929066
   0.02900794]
 [ 0.04893641  0.18120024  0.12434866 ... -0.26414418 -0.25138614
   0.06465693]
 [ 0.03905599 -0.00822297 -0.21407762 ... -0.16357931 -0.40986198
   0.81207806]]


In [154]:
# Inisialisasi model pretrained BERT
model_bert = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encode text kursus menjadi vektor
query_vectors = model_bert.encode(df_query['user_request'])

# Menampilkan matriks fitur dalam bentuk array
print("\nMatriks BERT:")
print(query_vectors)

# Hasil vektor dijadikan csv
df_vectors = pd.DataFrame(query_vectors)
df_vectors['user_request'] = df_query['user_request'].values
df_vectors.to_csv("query_vectors.csv", index=False)


Matriks BERT:
[[ 0.08833907 -0.30710128 -0.25401834 ...  0.28051138  0.19984742
  -0.18068369]
 [-0.5998062  -0.67357606  0.1257095  ... -0.32462266 -0.66892713
  -0.11698227]
 [-0.46614408  0.10662431 -0.01922847 ... -0.14281265 -0.30380914
   0.22047319]
 ...
 [ 0.03120843 -0.00218684 -0.31584018 ... -0.41084057 -0.27360767
  -0.04483977]
 [-0.254326    0.31289017 -0.3752971  ...  0.10745774 -0.0967809
   0.5974294 ]
 [ 0.4754377   0.03096627 -0.4709335  ... -0.5346767  -0.10527758
   0.06255824]]


# Similarity Calculation

Penggunaan cosine similarity dalam sistem rekomendasi ini bertujuan untuk mengukur tingkat kemiripan antara vektor teks, baik dari permintaan pengguna maupun dari deskripsi kursus, dalam ruang vektor berdimensi tinggi. Cosine similarity dipilih karena mampu mengukur seberapa dekat arah (orientasi) dua vektor, tanpa terpengaruh oleh besar (magnitude)-nya.

In [155]:
from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(course_vectors,query_vectors)
print("Matrix Cosine Similarity:")
print(similarity_matrix)

# Hasil cosine similarity dijadikan csv
df_similarity = pd.DataFrame(
    similarity_matrix,
    index=df_courses['title'],        
    columns=df_query['user_request'] 
)
df_similarity.to_csv("similarity_matrix.csv")

Matrix Cosine Similarity:
[[ 0.52329516  0.38607323  0.5461077  ...  0.508492    0.36028364
   0.23118378]
 [ 0.21565309  0.33278835  0.18829471 ...  0.23875025  0.29501092
   0.12995875]
 [ 0.27979887  0.23963313  0.22526003 ...  0.3351615   0.35270447
   0.22211441]
 ...
 [ 0.18594727  0.2196559   0.21229222 ...  0.1512848   0.18136531
  -0.03675438]
 [ 0.2024424   0.19781134  0.17145252 ...  0.24172086  0.2578013
   0.13583866]
 [ 0.2492495   0.24547264  0.26641303 ...  0.2489007   0.41909075
   0.1309461 ]]


# Build Recommender System

Sistem rekomendasi ini didasari oleh level_rank dan nilai similarity. Semakin rendah level_rank dan semakin tinggi similarity maka itulah yang terbaik.

In [156]:
def recommend(request_text, df_courses, similarity_matrix, df_query, top_n=5):
    request_index = df_query[df_query['user_request'] == request_text].index[0]

    course_similarities = similarity_matrix[request_index]
    
    temp_df = df_courses.copy()
    temp_df['similarity'] = course_similarities

    temp_df_sorted = temp_df.sort_values(by=['level_rank', 'similarity'], ascending=[True, False])
    
    recommended_courses = temp_df_sorted.head(top_n)['title'].tolist()

    return recommended_courses


In [157]:
for request_text in df_query['user_request'].tolist()[:3]:
    recommended_courses = recommend(request_text, df_courses, similarity_matrix, df_query)

    print(f"Recommended courses for request:\n\"{request_text}\"")
    for i, course_name in enumerate(recommended_courses, start=1):
        print(f"{i}. {course_name}")
    print()

Recommended courses for request:
"recommend detailed learning path online course teach python programming effectively"
1. C++ Fundamentals - A Complete Approach
2. Sustainable Urban Development
3. Hollywood: History, Industry, Art
4. Judaism Through Its Scriptures.
5. The What and How of Esports Management!

Recommended courses for request:
"im currently preparing career switch would like master neural network ground"
1. Negociación exitosa: Estrategias y habilidades esenciales (en español)
2. Analytics for Decision Making
3. JavaScript Basics
4. Introduction à l'Analyse du Cycle de Vie - de la pensée à la pratique
5. Python Data Structures

Recommended courses for request:
"best tool technique tutorial available online learn deep learning"
1. Inteligencia emocional para el liderazgo efectivo y colaboración
2. Gerencia estratégica: bases y principios
3. eLearning Course for Digital Skills for ICT Entrepreneurs in FCV Countries
4. Copyright Law in the Music Business
5. Digital Marketing

# Evaluation

In [158]:
def precision_at_k(predicted, ground_truth, k=3):
    predicted = predicted[:k]
    return len(set(predicted) & set(ground_truth)) / k

In [159]:
def recall_at_k(predicted, ground_truth, k=3):
    predicted = predicted[:k]
    return len(set(predicted) & set(ground_truth)) / len(ground_truth) if ground_truth else 0

In [160]:
request_text = "I would like to gain a deep understanding of pandas library and how it is used in real-world applications."
request_text = preprocessing(request_text)
ground_truth = ["IBM Data Science", "Python and Pandas for Data Engineering", "Analytics for Decision Making"]

predicted = recommend(request_text, df_courses, similarity_matrix, df_query, top_n=5)

prec = precision_at_k(predicted, ground_truth, k=5)
rec = recall_at_k(predicted, ground_truth, k=5)

print(f"Precision@5: {prec:.2f}")
print(f"Recall@5:    {rec:.2f}")

Precision@5: 0.20
Recall@5:    0.33


In [161]:
predicted

['The Threat of Nuclear Terrorism',
 'Introducción a la inversión bursátil',
 'Analytics for Decision Making',
 'JavaScript Basics',
 "Introduction à l'Analyse du Cycle de Vie - de la pensée à la pratique"]

Berdasarkan hasil evaluasi, dapat disimpulkan bahwa model rekomendasi yang digunakan saat ini masih belum optimal. Skor precision yang rendah menunjukkan bahwa sebagian besar kursus yang direkomendasikan tidak sesuai dengan kebutuhan pengguna, sementara skor recall yang belum maksimal mengindikasikan bahwa banyak kursus relevan yang belum berhasil dikenali oleh sistem. Hal ini menunjukkan bahwa representasi vektor dari teks atau strategi pemeringkatan masih perlu ditingkatkan agar model mampu menangkap makna permintaan pengguna secara lebih akurat.